# Lab | Random Forests

- For this lab, you will be using the CSV files provided in the files_for_lab folder.

In [36]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [45]:
numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
targets = pd.read_csv('target.csv')

### 1. Apply the Random Forests algorithm but this time only by upscaling the data using SMOTE.

### 2. Note that since SMOTE works on numerical data only, we will first encode the categorical variables in this case.

In [46]:
encoder = OneHotEncoder(drop='first').fit(categorical)
encoded_categorical = encoder.transform(categorical).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)

In [47]:
data = pd.concat([numerical, encoded_categorical, targets], axis = 1)

In [48]:
smote = SMOTE()
y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis=1)
X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

0    90569
1    90569
Name: TARGET_B, dtype: int64

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.25, random_state=0)

In [56]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.963696588274263


Is this score too perfect ?